<a href="https://colab.research.google.com/github/shdead56/Data-science/blob/main/ProyectoData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

WEB SCRAPING MANUAL

In [ ]:
import pandas as pd
import pickle
from string import ascii_uppercase as alphabet
all_tables = pd.read_html('https://web.archive.org/web/20221115040351/https://en.wikipedia.org/wiki/2022_FIFA_World_Cup')
for i in range(12,68,7): # 12 19 26
    print(i)
    df = all_tables[i]
    df.rename(columns={df.columns[1]:'Team'}, inplace=True)
    df.pop('Qualification')

dict_table = {}
for letter, i in zip(alphabet, range(12,68,7)): # A=11, B=18, ...
    df = all_tables[i]
    df.rename(columns={df.columns[1]:'Team'}, inplace=True)
    #df.pop('Qualification')
    dict_table[f'Group {letter}'] = df
with open('dict_table', 'wb') as output:
    pickle.dump(dict_table, output)



12
19
26
33
40
47
54
61


In [ ]:
dict_table.keys()
dict_table['Group A']

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Qatar (H),0,0,0,0,0,0,0,0
1,2,Ecuador,0,0,0,0,0,0,0,0
2,3,Senegal,0,0,0,0,0,0,0,0
3,4,Netherlands,0,0,0,0,0,0,0,0


WEB SCRAPING  BEATIFULSOUP

In [ ]:
#pip install bs4
#pip install requests
#pip install lxml
#pip install pandas
from bs4 import BeautifulSoup
import requests
import pandas as pd
years=[1930,1934,1938,1950,1954,1958,1962,1966,1970,1974,1978,
       1982,1986,1990,1994,1998,2002,2006,2010,2014,2018]
def get_partidos(year):
  web=f'https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup'
  response = requests.get(web)
  content = response.text
  soup = BeautifulSoup(content,'lxml')
  partidos = soup.find_all('div', class_= 'footballbox')

  local = []
  visita = []
  score = []
  for partido in partidos:
    local.append(partido.find ('th',class_='fhome').get_text())
    score.append(partido.find ('th',class_='fscore').get_text())
    visita.append(partido.find ('th',class_='faway').get_text())

  dict_futbol = {'local': local, 'score': score, 'visita': visita}
  df_futbol = pd.DataFrame(dict_futbol)
  df_futbol['year'] = year
  return df_futbol

def get_partidos_esp(web,year):
  response = requests.get(web)
  content = response.text
  soup = BeautifulSoup(content,'lxml')
  partidos = soup.find_all('div', class_= 'footballbox')

  local = []
  visita = []
  score = []
  for partido2 in partidos:
    local.append(partido2.find ('th',class_='fhome').get_text())
    score.append(partido2.find ('th',class_='fscore').get_text())
    visita.append(partido2.find ('th',class_='faway').get_text())

  dict_futbol2 = {'local': local, 'score': score, 'visita': visita}
  df_futbol2 = pd.DataFrame(dict_futbol2)
  df_futbol2['year'] = year
  return df_futbol2
#data historica
mundial = [get_partidos(year) for year in years]
df_mundial = pd.concat(mundial, ignore_index=True)
df_mundial.to_csv('mundiales_data.csv', index=False)

df_ficture = get_partidos_esp('https://web.archive.org/web/20221115040351/https://en.wikipedia.org/wiki/2022_FIFA_World_Cup','2022')
df_ficture.to_csv('fixture_2022.csv', index=False)


In [ ]:
import pandas as pd
df = pd.read_csv('mundiales_data.csv')
df2= df.loc[df['year'] == 1982]
df2 = df2.set_index(pd.Index(range(1, len(df2) + 1)))
print(df2)

              local         score             visita  year
1            Italy            0–0             Poland  1982
2             Peru            0–0           Cameroon  1982
3            Italy            1–1               Peru  1982
4           Poland            0–0           Cameroon  1982
5           Poland            5–1               Peru  1982
6            Italy            1–1           Cameroon  1982
7     West Germany            1–2            Algeria  1982
8            Chile            0–1            Austria  1982
9     West Germany            4–1              Chile  1982
10         Algeria            0–2            Austria  1982
11         Algeria            3–2              Chile  1982
12    West Germany            1–0            Austria  1982
13       Argentina            0–1            Belgium  1982
14         Hungary           10–1        El Salvador  1982
15       Argentina            4–1            Hungary  1982
16         Belgium            1–0        El Salvador  19

WEB SCRAPING SELENIUM

In [ ]:
#!pip install selenium
#!apt-get update
#!apt install chromium-chromedriver
!pip install kora -q
from kora.selenium import sel
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

#path = '/usr/lib/chromium-browser/chromedriver'
#ChromeOptions = webdriver.ChromeOptions()
#service = Service(executable_path=path)
#ChromeOptions = Options()
#ChromeOptions.add_argument("--headless")
#ChromeOptions.add_argument("--no-sandbox")
#ChromeOptions.add_argument("start-maximized")
#ChromeOptions.headless = True

# open it, go to a website, and get results
#driver = webdriver.Chrome(service=service,options=ChromeOptions)
def obtener_data(year):
  web = f'https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup'
  sel.get(web)
  partidos = sel.find_elements(by = 'xpath', value = '//td[@align="rigth"]/.. | //td[@style="text-align:rigth;"]/..')

  local = []
  score = []
  visita = []
  for partido in partidos:
    local.append(partido.find_element(by = 'xpath', value='./td[1]').text)
    score.append(partido.find_element(by = 'xpath', value='./td[2]').text)
    visita.append(partido.find_element(by = 'xpath', value='./td[3]').text)

  dict_futbol = {'local':local, 'score':score, 'visita':visita}
  df_futbol = pd.DataFrame(dict_futbol)
  df_futbol = pd.DataFrame(dict_futbol)
  df_futbol['year'] = year
  time.sleep(2)
  return df_futbol

years=[1930,1934,1938,1950,1954,1958,1962,1966,1970,1974,1978,
       1982,1986,1990,1994,1998,2002,2006,2010,2014,2018,2022]

mundiales = [obtener_data(year) for year in years]
df_mundiales = pd.concat(mundiales, ignore_index=True)
#driver.quit()
df_mundiales.to_csv('mundialesSE.csv', index=false)




LIMPIEZA Y TRANSFORMACION DE DATOS


In [ ]:
import pandas as pd
#LIMPIANDO DATA
df_historico = pd.read_csv('mundiales_data.csv')
df_fixture = pd.read_csv('fixture_2022.csv')
#df_missing = pd.read_csv('missing_data.csv')

df_fixture['local'] = df_fixture['local'].str.strip()
df_fixture['visita'] = df_fixture['visita'].str.strip()

df_historico['local'] = df_historico['local'].str.strip()
df_historico['visita'] = df_historico['visita'].str.strip()

#df_historico[df_historico['local'].isnull()]
#df_missing[df_missing['local'].isnull()]
#df_missing.dropna(inplace=True)
#df_historico.dropna(inplace=True)

#df_historico = pd.concat([df_historico, df_missing], ignore_index=True)
df_historico.drop_duplicates(inplace=True)
df_historico.sort_values('year', inplace=True)
#df_historico.reset_index(inplace=True, drop=False)
#df_historico.drop(['index'], axis=1, inplace=True)
df_historico


In [ ]:
#borrando partido con Walk over
delete_index = df_historico[df_historico['local'].str.contains('Sweden') &df_historico['visita'].str.contains('Austria')].index
df_historico.drop(index=delete_index, inplace=True)
df_historico['score'] = df_historico['score'].str.replace('[^\d–]', '', regex=True)
df_historico[['LocalGoals', 'VisitaGoals']] = df_historico['score'].str.split('–', expand=True)
df_historico.drop('score', axis=1, inplace=True)
#verificando si existen nulo
#df = df_historico
#filas_con_nulos = df[df.isnull().any(axis=1)]
#print(filas_con_nulos)
df_historico.loc[358, ['LocalGoals', 'VisitaGoals']] = ['3', '3']
df_historico = df_historico.astype({'LocalGoals': int, 'VisitaGoals':int, 'year': int})
df_historico['TotalGoals'] = df_historico['LocalGoals'] + df_historico['VisitaGoals']
df_historico
df_historico.to_csv('clean_mundiales.csv',index=False)
df_fixture.to_csv('clean_fixture.csv',index=False)


In [ ]:
years = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 1974,
         1978, 1982, 1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014,
         2018]

for year in years:
    print(year, len(df_historico[df_historico['year']==year]))

print(df_historico)

CREACION DEL MODELO

In [ ]:
#!pip install scipy
import pandas as pd
import pickle
from scipy.stats import poisson
dicc_tablas = pickle.load(open('dict_table','rb'))
df_historia_mundial = pd.read_csv('/content/clean_mundiales.csv')
df_fixture = pd.read_csv('clean_fixture.csv')

df_local = df_historia_mundial[['local', 'LocalGoals', 'VisitaGoals']]
df_visita = df_historia_mundial[['visita', 'LocalGoals', 'VisitaGoals']]

df_local = df_local.rename(columns={'local':'Team', 'LocalGoals': 'Anotados', 'VisitaGoals': 'Concedidos'})
df_visita = df_visita.rename(columns={'visita':'Team', 'LocalGoals': 'Concedidos', 'VisitaGoals': 'Anotados'})

df_ = pd.concat([df_local, df_visita], ignore_index=True).groupby(['Team']).mean()
df_fuerza_historica = pd.concat([df_local, df_visita], ignore_index=True).groupby(['Team']).mean()
df_fuerza_historica



,Anotados,Concedidos
Team,,
Algeria,1.000000,1.461538
Angola,0.333333,0.666667
Argentina,1.691358,1.148148
Australia,0.812500,1.937500
Austria,1.482759,1.620690
...,...,...
Uruguay,1.553571,1.321429
Wales,0.800000,0.800000
West Germany,2.112903,1.241935


In [ ]:
def predict_points(local, visita):
    if local in df_fuerza_historica.index and visita in df_fuerza_historica.index:
        # goals_scored * goals_conceded
        lamb_local = df_fuerza_historica.at[local,'Anotados'] * df_fuerza_historica.at[visita,'Concedidos']
        lamb_visita = df_fuerza_historica.at[visita,'Anotados'] * df_fuerza_historica.at[local,'Concedidos']
        prob_local, prob_visita, prob_draw = 0, 0, 0
        for x in range(0,11): #number of goals home team
            for y in range(0, 11): #number of goals away team
                p = poisson.pmf(x, lamb_local) * poisson.pmf(y, lamb_visita)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_local += p
                else:
                    prob_visita += p

        points_local = 3 * prob_local + prob_draw
        points_visita = 3 * prob_visita + prob_draw
        return (points_local, points_visita)
    else:
        return (0, 0)

In [ ]:
print(predict_points('England', 'United States'))
print(predict_points('Argentina', 'Mexico'))
print(predict_points('Qatar (H)', 'Ecuador'))

(2.2356147635326007, 0.5922397535606193)
(2.3129151525530505, 0.5378377125059863)
(0, 0)


In [ ]:
df_fixture_group_48 = df_fixture[:48].copy()
df_fixture_knockout = df_fixture[48:56].copy()
df_fixture_quarter = df_fixture[56:60].copy()
df_fixture_semi = df_fixture[60:62].copy()
df_fixture_final = df_fixture[62:].copy()

In [ ]:
for group in dict_table:
    teams_in_group = dict_table[group]['Team'].values
    df_fixture_group_6 = df_fixture_group_48[df_fixture_group_48['local'].isin(teams_in_group)]
    for index, row in df_fixture_group_6.iterrows():
        home, away = row['local'], row['visita']
        points_home, points_away = predict_points(home, away)
        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away

    dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index()
    dict_table[group] = dict_table[group][['Team', 'Pts']]
    dict_table[group] = dict_table[group].round(0)

In [ ]:
dict_table['Group E']

,Team,Pts
0,Germany,7.0
1,Spain,5.0
2,Japan,3.0
3,Costa Rica,2.0


    **OCATAVOS

In [ ]:
for group in dict_table:
    group_winner = dict_table[group].loc[0, 'Team']
    runners_up = dict_table[group].loc[1, 'Team']
    df_fixture_knockout.replace({f'Winners {group}':group_winner,
                                 f'Runners-up {group}':runners_up}, inplace=True)

def get_winner(df_fixture_updated):
    for index, row in df_fixture_updated.iterrows():
        home, away = row['local'], row['visita']
        points_home, points_away = predict_points(home, away)
        if points_home > points_away:
            winner = home
        else:
            winner = away
        df_fixture_updated.loc[index, 'winner'] = winner
    return df_fixture_updated

get_winner(df_fixture_knockout)

,local,score,visita,year,winner
48,Netherlands,Match 49,Wales,2022,Netherlands
49,Argentina,Match 50,Denmark,2022,Argentina
50,France,Match 52,Poland,2022,France
51,England,Match 51,Senegal,2022,England
52,Germany,Match 53,Belgium,2022,Germany
53,Brazil,Match 54,Uruguay,2022,Brazil
54,Croatia,Match 55,Spain,2022,Spain
55,Portugal,Match 56,Switzerland,2022,Portugal


    CUARTOS

In [ ]:
def update_table(df_fixture_round_1, df_fixture_round_2):
    for index, row in df_fixture_round_1.iterrows():
        winner = df_fixture_round_1.loc[index, 'winner']
        match = df_fixture_round_1.loc[index, 'score']
        df_fixture_round_2.replace({f'Winners {match}':winner}, inplace=True)
    df_fixture_round_2['winner'] = '?'
    return df_fixture_round_2

update_table(df_fixture_knockout, df_fixture_quarter)
get_winner(df_fixture_quarter)

,local,score,visita,year,winner
56,Germany,Match 58,Brazil,2022,Brazil
57,Netherlands,Match 57,Argentina,2022,Netherlands
58,Spain,Match 60,Portugal,2022,Portugal
59,England,Match 59,France,2022,France


    SEMIFINAL

In [ ]:
update_table(df_fixture_quarter, df_fixture_semi)
get_winner(df_fixture_semi)

,local,score,visita,year,winner
60,Netherlands,Match 61,Brazil,2022,Brazil
61,France,Match 62,Portugal,2022,France


    Final

In [ ]:
update_table(df_fixture_semi, df_fixture_final)
get_winner(df_fixture_final)

,local,score,visita,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,Losers Match 62
63,Brazil,Match 64,France,2022,Brazil
